# Cox model

We will use the cox model with the result of the feature selection done in Lasso (part done in R)

In [3]:
# we impot tools where we have defined usefull functions 
import tools
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter

In [7]:
trainingSet=pd.read_csv('mergedStandardizedTrainingSet.csv').set_index('PatientID')
testingSet=pd.read_csv('mergedStandardizedTestingSet.csv').set_index('PatientID')

In [41]:
# We initialise the cox model with a penalizer. This will allow regularizating the coeffecicients
cph = CoxPHFitter(penalizer=0.2)

# we fit the model with the feature selected in R thanks to Lasso

cph.fit(trainingSet[['Nstage', 'SourceDataset', 'original_shape_VoxelVolume', 'original_firstorder_Maximum'
                                        , 'original_firstorder_Mean'
                                        , 'original_glcm_Idn', 'original_glcm_Idm', 'original_glrlm_RunPercentage'
                                        , 'age', 'original_glcm_ClusterProminence'
                                        , 'SurvivalTime', 'Event']]
        , duration_col='SurvivalTime', event_col='Event')

# we predict the median
predictionFromTraining=cph.predict_median(trainingSet)
# When the rediction is inf, we set it to the max survival time of the training set
predictionFromTraining=predictionFromTraining.replace([np.inf, -np.inf], max(trainingSet.SurvivalTime))
# We change the name the column 0.5
predictionFromTraining.rename(columns = {0.5:'SurvivalTime'}, inplace = True)
# We get the metric '' thanks to the provided function
print(tools.getMetric (predictionFromTraining, trainingSet[['SurvivalTime','Event']]))
# we do the same operations for the Testing set
predictionFromTesting=cph.predict_median(testingSet)
predictionFromTesting=predictionFromTesting.replace([np.inf, -np.inf], 3500)
predictionFromTesting.rename(columns = {0.5:'SurvivalTime'}, inplace = True)
# we create a column 'nan' for the submission
predictionFromTesting['Event']='nan'
predictionFromTesting.to_csv('5thAttemptUsingPythonLifeline.csv')

0.7217002024801563


In [101]:
test=pd.read_csv('test.csv')
test.rename(columns = {'V1':'SurvivalTime'}, inplace = True)
test['PatientID']=trainingSet.index
test=test.set_index('PatientID')
test.pop('Unnamed: 0')
test['Event']='nan'
test.head()
test.to_csv('1stAttemptWithRF.csv')

In [99]:
print(tools.getMetric (test, trainingSet[['SurvivalTime','Event']]))
test.to_csv('1stAttemptWithRF.csv')

0.8112028600351774


,SurvivalTime,Event
PatientID,,
202,2408,NaN
371,642,NaN
246,539,NaN
240,1295,NaN
284,865,NaN
348,573,NaN
384,265,NaN
244,1369,NaN
100,325,NaN
